In [1]:
from google.cloud import bigquery
from google.oauth2 import service_account
import pandas as pd

In [2]:
# CREDS = 'is3107-418808-5bca2b8d29d0.json'
CREDS = 'is3107-418011-f63573e5e1f3.json'
credentials = service_account.Credentials.from_service_account_file(CREDS)
client = bigquery.Client(credentials=credentials)
job_config = bigquery.QueryJobConfig()

In [3]:
dataset_id='is3107-418011.is3107'       
print(list(client.list_datasets(credentials.project_id)))
tables = client.list_tables(dataset_id)  
for table in tables:
    print(table.table_id)

Address
Property
Surroundings
Transaction
resale_price
testing
testing2


In [75]:
df = pd.read_csv('ETL/01_Data_Collection/01_dataset/processed_data/filtered_df3.csv')
df.head()

/var/folders/l5/0p3sf6hj6sqd3k5fbpl4prrw0000gn/T/ipykernel_30060/860810733.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('ETL/01_Data_Collection/01_dataset/processed_data/filtered_df3.csv')


,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,...,school_type,bus_stop_proximity,mrt_proximity,pri_sch_proximity,sec_sch_proximity,latitude,longitude,inflation,normalized_resale_price,index
0,2000-01,ANG MO KIO,3 ROOM,170,ANG MO KIO AVE 4,07 TO 09,69.0,Improved,1986,147000.0,...,elite,3-5 minutes,more than 15 minutes,3-5 minutes,3-5 minutes,1.374001,103.836432,1.322265,194372.998483,0
1,2000-02,ANG MO KIO,3 ROOM,170,ANG MO KIO AVE 4,07 TO 09,60.0,Improved,1986,130000.0,...,elite,3-5 minutes,more than 15 minutes,3-5 minutes,3-5 minutes,1.374001,103.836432,1.322265,171894.488454,1
2,2000-03,ANG MO KIO,2 ROOM,170,ANG MO KIO AVE 4,04 TO 06,45.0,Improved,1986,105000.0,...,elite,3-5 minutes,more than 15 minutes,3-5 minutes,3-5 minutes,1.374001,103.836432,1.322265,138837.856059,2
3,2000-05,ANG MO KIO,2 ROOM,170,ANG MO KIO AVE 4,04 TO 06,45.0,Improved,1986,125000.0,...,elite,3-5 minutes,more than 15 minutes,3-5 minutes,3-5 minutes,1.374001,103.836432,1.301036,162629.480554,3
4,2000-06,ANG MO KIO,2 ROOM,170,ANG MO KIO AVE 4,07 TO 09,45.0,Improved,1986,117000.0,...,elite,3-5 minutes,more than 15 minutes,3-5 minutes,3-5 minutes,1.374001,103.836432,1.301036,152221.193798,4


In [76]:
mixed_type_columns = df.select_dtypes(include='object').columns

if mixed_type_columns.empty:
    print("No columns with mixed data types.")
else:
    print("Columns with mixed data types:")
    print(mixed_type_columns)
    
df[mixed_type_columns] = df[mixed_type_columns].astype(str)

Columns with mixed data types:
Index(['month', 'town', 'flat_type', 'block', 'street_name', 'storey_range',
       'flat_model', 'address', 'planning_area', 'mrt_name', 'bus_stop_name',
       'pri_sch_name', 'sec_sch_name', 'flat_type_model', 'school_type',
       'bus_stop_proximity', 'mrt_proximity', 'pri_sch_proximity',
       'sec_sch_proximity'],
      dtype='object')


In [7]:
len(df.columns)

50

In [86]:
property_id = ['flat_type', 'flat_model', 'flat_type_model',
                  'floor_area_sqm', 'lease_commence_date', 'storey_range',
                  'avg_storey_range', 'total_dwelling_units']
address_id = ['postal','address', 'town', 'block', 'street_name', 
                          'planning_area', 'latitude', 'longitude']
surroundings_id = ['commercial', 'market_hawker', 'multistorey_carpark', 'precinct_pavilion',
                 'mall_nearest_distance', 'hawker_nearest_distance',
                               'hawker_food_stalls', 'hawker_market_stalls', 'mrt_nearest_distance', 'mrt_name', 'bus_interchange', 'mrt_interchange', 'bus_stop_nearest_distance', 'bus_stop_name',
                               'pri_sch_nearest_distance', 'pri_sch_name', 'vacancy', 'pri_sch_affiliation',
                               'sec_sch_nearest_dist', 'sec_sch_name', 'cutoff_point', 'affiliation', 'school_type',
                               'bus_stop_proximity', 'mrt_proximity', 'pri_sch_proximity', 'sec_sch_proximity']
transaction_id = ['month', 'year', 'quarter', 'inflation', 'normalized_resale_price']


property_df = df[property_id].copy().drop_duplicates()
property_df["property_id"] = [i for i in range(1 , len(property_df) + 1)]


address_df = df[address_id].copy().drop_duplicates()
address_df["address_id"] = [i for i in range(1 , len(address_df) + 1)]

surroundings_df = df[surroundings_id].copy().drop_duplicates()
surroundings_df["surroundings_id"] = [i for i in range(1 , len(surroundings_df) + 1)]

transaction_df = df[transaction_id].copy().drop_duplicates()
transaction_df["transaction_id"] = [i for i in range(1 , len(transaction_df) + 1)]

resale_data = df.copy()

In [104]:
property_df

,flat_type,flat_model,flat_type_model,floor_area_sqm,lease_commence_date,storey_range,avg_storey_range,total_dwelling_units,property_id
0,3 ROOM,Improved,3 ROOM Improved,69.0,1986,07 TO 09,8.0,179,1
1,3 ROOM,Improved,3 ROOM Improved,60.0,1986,07 TO 09,8.0,179,2
2,2 ROOM,Improved,2 ROOM Improved,45.0,1986,04 TO 06,5.0,179,3
4,2 ROOM,Improved,2 ROOM Improved,45.0,1986,07 TO 09,8.0,179,4
5,3 ROOM,Improved,3 ROOM Improved,69.0,1986,04 TO 06,5.0,179,5
...,...,...,...,...,...,...,...,...,...
717031,4 ROOM,Model A,4 ROOM Model A,94.0,2017,13 TO 15,14.0,110,105736
717033,5 ROOM,Improved,5 ROOM Improved,114.0,2017,01 TO 03,2.0,110,105737
717037,5 ROOM,Improved,5 ROOM Improved,114.0,2017,04 TO 06,5.0,110,105738
717048,4 ROOM,Model A,4 ROOM Model A,94.0,2017,07 TO 09,8.0,110,105739


In [106]:
property_df["flat_model"][0]

'Improved'

In [105]:
property_id

['flat_type',
 'flat_model',
 'flat_type_model',
 'floor_area_sqm',
 'lease_commence_date',
 'storey_range',
 'avg_storey_range',
 'total_dwelling_units']

In [87]:
# Merge to map IDs back to the main DataFrame
resale_data = resale_data.merge(property_df, on=property_id, how='left')
resale_data = resale_data.merge(address_df, on=address_id, how='left')
resale_data = resale_data.merge(surroundings_df, on=surroundings_id, how='left')
resale_data = resale_data.merge(transaction_df, on=transaction_id, how='left')

all_ids = set(property_id + address_id + surroundings_id + transaction_id)
resale_data.drop(columns=list(all_ids), inplace=True)

display(resale_data)

,resale_price,index,property_id,address_id,surroundings_id,transaction_id
0,147000.0,0,1,1,1,1
1,130000.0,1,2,1,1,2
2,105000.0,2,3,1,1,3
3,125000.0,3,3,1,1,4
4,117000.0,4,4,1,1,5
...,...,...,...,...,...,...
717064,758000.0,717064,105738,9013,8992,97958
717065,795000.0,717065,105733,9013,8992,155177
717066,680000.0,717066,105735,9013,8992,74794
717067,818000.0,717067,105733,9013,8992,30917


In [110]:
unique_pair = df[['flat_model', 'town']].apply(lambda x: x.str.lower()).drop_duplicates()

# Display the resulting DataFrame with unique pairs
unique_pair

,flat_model,town
0,improved,ang mo kio
168,adjoined flat,ang mo kio
342,new generation,ang mo kio
12035,model a,ang mo kio
15608,standard,ang mo kio
...,...,...
711162,dbss,hougang
712516,dbss,clementi
713666,premium apartment loft,queenstown
713670,premium apartment,queenstown


In [62]:
resale_data = df.copy()
resale_data[property_id] = resale_data['index'].map(property_df.set_index(property_id)['property_id'])
resale_data["address_id"] = resale_data['index'].map(address_df.set_index('index')['address_id'])
resale_data["surroundings_id"] = resale_data['index'].map(surroundings_df.set_index('index')['surroundings_id'])
resale_data["transaction_id"] = resale_data['index'].map(transaction_df.set_index('index')['transaction_id'])

all_ids = set(property_id + address_id + surroundings_id + transaction_id)
resale_data.drop(columns=list(all_ids), inplace=True)

# Displaying the DataFrame
display(resale_data)

ValueError: Columns must be same length as key

In [33]:
all_ids = property_id + address_id + surroundings_id + transation_id

# Create a dictionary to count occurrences of each value
id_counts = {}
for id_value in all_ids:
    if id_value in id_counts:
        id_counts[id_value] += 1
    else:
        id_counts[id_value] = 1

# Find duplicate values
duplicate_values = [id_value for id_value, count in id_counts.items() if count > 1]

print("Duplicate values:", duplicate_values)

Duplicate values: ['index']


In [83]:
property = df[['index', 'flat_type', 'flat_model', 'flat_type_model',
                  'floor_area_sqm', 'lease_commence_date', 'storey_range',
                  'avg_storey_range', 'total_dwelling_units','vacancy']]

# Drop duplicate rows based on the 'index' column
property = property.drop_duplicates()

property_duplicate_rows = property[property.duplicated()]

print(len(property))

property_duplicate_rows


717069


,index,flat_type,flat_model,flat_type_model,floor_area_sqm,lease_commence_date,storey_range,avg_storey_range,total_dwelling_units,vacancy


In [51]:
len(address.columns)

8

In [77]:
address = df[['postal', 'address', 'town', 'block', 'street_name', 
                          'planning_area', 'latitude', 'longitude',]]

# Drop duplicate rows based on the 'index' column
address = address.drop_duplicates()

address_duplicate_rows = address[address.duplicated()]

address_duplicate_rows

,postal,address,town,block,street_name,planning_area,latitude,longitude


In [53]:
len(surroundings.columns)

29

In [78]:
surroundings = df[['postal', 'commercial', 'market_hawker', 'multistorey_carpark', 'precinct_pavilion',
                               'mall_nearest_distance', 'hawker_nearest_distance',
                               'hawker_food_stalls', 'hawker_market_stalls', 'mrt_nearest_distance', 'mrt_name',
                               'bus_interchange', 'mrt_interchange', 'bus_stop_nearest_distance', 'bus_stop_name',
                               'pri_sch_nearest_distance', 'pri_sch_name',  'pri_sch_affiliation',
                               'sec_sch_nearest_dist', 'sec_sch_name', 'cutoff_point', 'affiliation', 'school_type',
                               'bus_stop_proximity', 'mrt_proximity', 'pri_sch_proximity', 'sec_sch_proximity']]

# Drop duplicate rows based on the 'index' column
surroundings = surroundings.drop_duplicates()

surroundings_duplicate_rows = surroundings[surroundings.duplicated()]

surroundings_duplicate_rows

,postal,commercial,market_hawker,multistorey_carpark,precinct_pavilion,mall_nearest_distance,hawker_nearest_distance,hawker_food_stalls,hawker_market_stalls,mrt_nearest_distance,...,pri_sch_affiliation,sec_sch_nearest_dist,sec_sch_name,cutoff_point,affiliation,school_type,bus_stop_proximity,mrt_proximity,pri_sch_proximity,sec_sch_proximity


In [55]:
len(transaction.columns)

8

In [79]:
transaction = df[['index','postal', 'month', 'resale_price', 'year', 'quarter', 'inflation', 'normalized_resale_price']]

# Drop duplicate rows based on the 'index' column
transaction = transaction.drop_duplicates()

transaction_duplicate_rows = transaction[transaction.duplicated()]

transaction_duplicate_rows


,index,postal,month,resale_price,year,quarter,inflation,normalized_resale_price


In [85]:
output = pd.merge(transaction, property, on="index")
output = pd.merge(output, address, on="postal")
output = pd.merge(output, surroundings, on="postal")
len(output)

723516

In [25]:
project_id = "is3107-418011"

# Dataset ID and table name
dataset_id = "is3107"  # You have nested dataset, so specify only the innermost dataset
table_id = "resale_price"   # Table ID within the specified dataset

# Upload DataFrame to BigQuery
df.to_gbq(destination_table=f"{project_id}.{dataset_id}.{table_id}", project_id=project_id, if_exists="replace")

100%|██████████| 1/1 [00:00<00:00, 2091.92it/s]


In [72]:
# Dataset ID and table name
table_id = "Property"   # Table ID within the specified dataset

# Upload DataFrame to BigQuery
property.to_gbq(destination_table=f"{project_id}.{dataset_id}.{table_id}", project_id=project_id, if_exists="replace")

100%|██████████| 1/1 [00:00<00:00, 32263.88it/s]


In [73]:
# Dataset ID and table name
table_id = "Address"   # Table ID within the specified dataset

# Upload DataFrame to BigQuery
address.to_gbq(destination_table=f"{project_id}.{dataset_id}.{table_id}", project_id=project_id, if_exists="replace")

100%|██████████| 1/1 [00:00<00:00, 11096.04it/s]


In [74]:
# Dataset ID and table name
table_id = "Surroundings"   # Table ID within the specified dataset

# Upload DataFrame to BigQuery
surroundings.to_gbq(destination_table=f"{project_id}.{dataset_id}.{table_id}", project_id=project_id, if_exists="replace")

100%|██████████| 1/1 [00:00<00:00, 8630.26it/s]


In [75]:
# Dataset ID and table name
table_id = "Transaction"   # Table ID within the specified dataset

# Upload DataFrame to BigQuery
transaction.to_gbq(destination_table=f"{project_id}.{dataset_id}.{table_id}", project_id=project_id, if_exists="replace")

100%|██████████| 1/1 [00:00<00:00, 36792.14it/s]


In [16]:
def query_table_from_bq(table_name):
    CREDS = 'is3107-418011-f63573e5e1f3.json'
    credentials = service_account.Credentials.from_service_account_file(CREDS)
    client = bigquery.Client(credentials=credentials)
    job_config = bigquery.QueryJobConfig()

    # Set the destination table
    table = client.dataset(dataset_id='is3107-418011.is3107').table(table_id=f"is3107-418011.is3107.{table_name}")
    job_config.destination = table
    query = f"SELECT * FROM `is3107-418011.is3107.{table_name}`"
    return client.query(query).to_dataframe()

In [ ]:
def query_table_from_bq():
    CREDS = 'is3107-418011-f63573e5e1f3.json'
    credentials = service_account.Credentials.from_service_account_file(CREDS)
    client = bigquery.Client(credentials=credentials)
    # job_config = bigquery.QueryJobConfig()
    #
    # # Set the destination table
    # table = client.dataset(dataset_id='is3107-418011.is3107').table(table_id=f"is3107-418011.is3107.Transation_new")
    # job_config.destination = table

    query = """
        SELECT *
        FROM `is3107-418011.is3107.resale_data`
        """

    return client.query(query).to_dataframe()

In [100]:
query_table_from_bq()

,resale_price,flat_model,town,date
0,147000.0,improved,ang mo kio,2000-01-31
1,130000.0,improved,ang mo kio,2000-02-29
2,105000.0,improved,ang mo kio,2000-03-31
3,125000.0,improved,ang mo kio,2000-05-31
4,117000.0,improved,ang mo kio,2000-06-30
...,...,...,...,...
717064,758000.0,improved,tampines,2023-04-30
717065,795000.0,improved,tampines,2023-07-31
717066,680000.0,model a,tampines,2023-09-30
717067,818000.0,improved,tampines,2023-10-31


In [101]:
CREDS = 'is3107-418011-f63573e5e1f3.json'
credentials = service_account.Credentials.from_service_account_file(CREDS)
client = bigquery.Client(credentials=credentials)

In [114]:
def query_table_from_bq3(town_type):
    # Assuming `flat_model_type` is a string containing the model type to filter for
    # Using parameterized queries for safety and efficiency
    query = """
            SELECT 
                LOWER(p.flat_model) as flat_model, 
                LOWER(a.town) as town, 
                AVG(d.resale_price) as avg_resale_price,
                LAST_DAY(PARSE_DATE('%Y-%m', t.month)) as date
            FROM `is3107-418011.is3107.resale_data` d
            LEFT JOIN `is3107-418011.is3107.Property_new` p ON d.property_id = p.property_id
            LEFT JOIN `is3107-418011.is3107.Address_new` a ON d.address_id = a.address_id
            LEFT JOIN `is3107-418011.is3107.Transaction_new` t ON d.transaction_id = t.transaction_id
            WHERE LOWER(a.town) = @town
            GROUP BY flat_model, town, date
        """

    job_config = bigquery.QueryJobConfig(
        query_parameters=[
            bigquery.ScalarQueryParameter("town", "STRING", town_type.lower())
        ]
    )
    
    # Execute the query with the specified parameter
    return client.query(query, job_config=job_config).to_dataframe()

temp3 = query_table_from_bq3("tampines")
temp3

,flat_model,town,avg_resale_price,date
0,improved,tampines,368315.192982,2000-01-31
1,improved,tampines,380169.491525,2000-07-31
2,improved,tampines,356524.418605,2000-10-31
3,model a,tampines,251089.592593,2000-12-31
4,improved,tampines,345763.285714,2001-03-31
...,...,...,...,...
2389,dbss,tampines,625000.000000,2020-11-30
2390,dbss,tampines,699000.000000,2022-04-30
2391,dbss,tampines,800000.000000,2020-04-30
2392,dbss,tampines,610000.000000,2023-10-31


In [111]:
def query_table_from_bq2():
    query = """
        SELECT 
            LOWER(p.flat_model) as flat_model, 
            LOWER(a.town) as town, 
            AVG(d.resale_price) as avg_resale_price,
            LAST_DAY(PARSE_DATE('%Y-%m', t.month)) as date
        FROM `is3107-418011.is3107.resale_data` d
        LEFT JOIN `is3107-418011.is3107.Property_new` p ON d.property_id = p.property_id
        LEFT JOIN `is3107-418011.is3107.Address_new` a ON d.address_id = a.address_id
        LEFT JOIN `is3107-418011.is3107.Transaction_new` t ON d.transaction_id = t.transaction_id
        GROUP BY flat_model, town, date
    """

    return client.query(query).to_dataframe()
temp2 = query_table_from_bq2()

In [112]:
temp2

,flat_model,town,avg_resale_price,date
0,improved,ang mo kio,3.605714e+05,2000-01-31
1,improved,ang mo kio,3.230000e+05,2000-02-29
2,improved,ang mo kio,3.923056e+05,2000-03-31
3,improved,ang mo kio,3.651250e+05,2000-05-31
4,improved,ang mo kio,3.309286e+05,2000-06-30
...,...,...,...,...
46814,premium apartment,queenstown,8.500000e+05,2023-05-31
46815,premium apartment,queenstown,7.300000e+05,2023-06-30
46816,premium apartment loft,queenstown,1.092000e+06,2023-06-30
46817,premium apartment loft,queenstown,1.038000e+06,2023-08-31


In [102]:
def query_table_from_bq():
    # job_config = bigquery.QueryJobConfig()
    #
    # # Set the destination table
    # table = client.dataset(dataset_id='is3107-418011.is3107').table(table_id=f"is3107-418011.is3107.Transation_new")
    # job_config.destination = table

    query = """
        SELECT d.resale_price, LOWER(p.flat_model) as flat_model, LOWER(a.town) as town, LAST_DAY(PARSE_DATE('%Y-%m', t.month)) as date
        FROM `is3107-418011.is3107.resale_data` d
        LEFT JOIN `is3107-418011.is3107.Property_new` p ON d.property_id = p.property_id
        LEFT JOIN `is3107-418011.is3107.Address_new` a ON d.address_id = a.address_id
        LEFT JOIN `is3107-418011.is3107.Transaction_new` t ON d.transaction_id = t.transaction_id
        """

    return client.query(query).to_dataframe()

In [103]:
temp = query_table_from_bq()

In [94]:
temp.head()

,resale_price,flat_model,town,date
0,147000.0,improved,ang mo kio,2000-01-31
1,130000.0,improved,ang mo kio,2000-02-29
2,105000.0,improved,ang mo kio,2000-03-31
3,125000.0,improved,ang mo kio,2000-05-31
4,117000.0,improved,ang mo kio,2000-06-30


In [97]:
dict2 = {"flat_model":"new generation",
"town":"punggol"}
temp[temp["town"] == dict2["town"]]

,resale_price,flat_model,town,date
629384,292800.0,improved,punggol,2005-04-30
629385,295000.0,improved,punggol,2006-07-31
629386,304000.0,improved,punggol,2006-08-31
629387,315000.0,improved,punggol,2007-04-30
629388,258000.0,model a,punggol,2007-05-31
...,...,...,...,...
716764,675000.0,improved,punggol,2021-10-31
716765,510000.0,model a,punggol,2022-01-31
716766,688000.0,improved,punggol,2022-04-30
716767,545000.0,model a,punggol,2023-09-30
